This dataset describes 5-star rating and free-text tagging activity from [MovieLens](http://movielens.org), a movie recommendation service. It contains 100836 ratings and 3683 tag applications across 9742 movies. These data were created by 610 users between March 29, 1996 and September 24, 2018. This dataset was generated on September 26, 2018.

Users were selected at random for inclusion. All selected users had rated at least 20 movies. No demographic information is included. Each user is represented by an id, and no other information is provided.

The data are contained in the files 'links.csv', 'movies.csv', 'ratings.csv' and 'tags.csv'. More details about the contents and use of all these files follows:

- User Ids: Unique and anonymized.
- Movies Ids: Only movies with at least one rating. 

In [109]:
# Let's include general porpuses libraries 

import pandas as pd
import numpy as np                     # For mathematical calculations
import seaborn as sns                  # For data visualization
import matplotlib.pyplot as plt        # For plotting graphs
%matplotlib inline

Let's check one by one all the data we have 

In [110]:
dfratings = pd.read_csv('ratings.csv')
dfratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


Ratings are made in a 5-star scale, with half-star increments
Timestamp data is the time when the movie was rated (Timestamps represent seconds since midnight Coordinated Universal Time (UTC) of January 1, 1970)

In [111]:
dfratings.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [112]:
dfratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


Let's translate the timestamp in a datetime type in python

In [113]:
dfratings['timestamp'].values

array([ 964982703,  964981247,  964982224, ..., 1494273047, 1493846352,
       1493846415], dtype=int64)

In [114]:
from datetime import datetime

dates = []

for ts in dfratings['timestamp'].values:
    dates.append((datetime.fromtimestamp((ts))))
    
dfratings['dates'] = dates

In [60]:
dfratings=dfratings.drop('timestamp', axis=1)
dfratings.head()

,userId,movieId,rating,dates
0,1,1,4.0,2000-07-30 20:45:03
1,1,3,4.0,2000-07-30 20:20:47
2,1,6,4.0,2000-07-30 20:37:04
3,1,47,5.0,2000-07-30 21:03:35
4,1,50,5.0,2000-07-30 20:48:51


In [61]:
dftags = pd.read_csv('tags.csv')
dftags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


This table has free-text taggind done by users. Also claissified by userId and movieId. The meaning, value, and purpose of a particular tag is determined by each user.

In [62]:
dftags = dftags.drop('timestamp', axis=1)

In [63]:
dftags.isnull().sum()

userId     0
movieId    0
tag        0
dtype: int64

In [64]:
dftags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   userId   3683 non-null   int64 
 1   movieId  3683 non-null   int64 
 2   tag      3683 non-null   object
dtypes: int64(2), object(1)
memory usage: 86.4+ KB


In [65]:
dfmovies = pd.read_csv('movies.csv')
dfmovies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


Title includes the title of the movie and the year of release. They are also classified in different Genres:

In [66]:
dfmovies['genres'].value_counts()

Drama                                     1053
Comedy                                     946
Comedy|Drama                               435
Comedy|Romance                             363
Drama|Romance                              349
                                          ... 
Action|Fantasy|Horror|Mystery|Thriller       1
Animation|Children|Drama|Fantasy             1
Crime|Drama|War                              1
Action|Animation|Crime|Drama                 1
Adventure|Horror|Sci-Fi                      1
Name: genres, Length: 951, dtype: int64

They belong to any of the following categories or all the possible combination between them
* Action
* Adventure
* Animation
* Children's
* Comedy
* Crime
* Documentary
* Drama
* Fantasy
* Film-Noir
* Horror
* Musical
* Mystery
* Romance
* Sci-Fi
* Thriller
* War
* Western

In [67]:
dfmovies.isnull().sum()

movieId    0
title      0
genres     0
dtype: int64

In [68]:
dfmovies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


let's separate title and year of release in two different columns:

In [69]:
#spliter la colonne "Title' en deux colonnes 'title' et 'year'
df_temp = pd.DataFrame(dfmovies.title.str.split('\(',1).tolist(),
columns = ['title','year'])
dfmovies['title'] = df_temp['title']
dfmovies['year'] = df_temp['year']

In [70]:
#supprimer le parenthese ')' de la colonne 'year'
dfmovies['year'] = dfmovies['year'].str.replace(r'\)', '')
dfmovies.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


We separate the genres in individual columns, if a movie belongs to a speficic genre, the column is encoded with 1, otherwise 0

In [71]:
dfmovies['Action']=dfmovies['genres'].apply(lambda x: 1 if "Action" in x else 0)
dfmovies['Adventure']=dfmovies['genres'].apply(lambda x: 1 if "Adventure" in x else 0)
dfmovies['Animation']=dfmovies['genres'].apply(lambda x: 1 if "Animation" in x else 0)
dfmovies['Children']=dfmovies['genres'].apply(lambda x: 1 if "Children" in x else 0)
dfmovies['Comedy']=dfmovies['genres'].apply(lambda x: 1 if "Comedy" in x else 0)
dfmovies['Crime']=dfmovies['genres'].apply(lambda x: 1 if "Crime" in x else 0)
dfmovies['Documentary']=dfmovies['genres'].apply(lambda x: 1 if "Documentary" in x else 0)
dfmovies['Drama']=dfmovies['genres'].apply(lambda x: 1 if "Drama" in x else 0)
dfmovies['Fantasy']=dfmovies['genres'].apply(lambda x: 1 if "Fantasy" in x else 0)
dfmovies['Film-Noir']=dfmovies['genres'].apply(lambda x: 1 if "Film-Noir" in x else 0)
dfmovies['Horror']=dfmovies['genres'].apply(lambda x: 1 if "Horror" in x else 0)
dfmovies['Musical']=dfmovies['genres'].apply(lambda x: 1 if "Musical" in x else 0)
dfmovies['Mystery']=dfmovies['genres'].apply(lambda x: 1 if "Mystery" in x else 0)
dfmovies['Romance']=dfmovies['genres'].apply(lambda x: 1 if "Romance" in x else 0)
dfmovies['Sci-Fi']=dfmovies['genres'].apply(lambda x: 1 if "Sci-Fi" in x else 0)
dfmovies['Thriller']=dfmovies['genres'].apply(lambda x: 1 if "Thriller" in x else 0)
dfmovies['War']=dfmovies['genres'].apply(lambda x: 1 if "War" in x else 0)
dfmovies['Western']=dfmovies['genres'].apply(lambda x: 1 if "Western" in x else 0)

In [72]:
dfmovies=dfmovies.drop('genres', axis=1)
dfmovies.head()

,movieId,title,year,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story,1995,0,1,1,1,1,0,0,...,1,0,0,0,0,0,0,0,0,0
1,2,Jumanji,1995,0,1,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men,1995,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale,1995,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II,1995,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
dflinks = pd.read_csv('links.csv')
dflinks.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


imdbId is the identifier of the movie in imdbId. For possible connection with http://www.imdb.com, https://www.imdb.com/title/tt0 + imdbId

https://www.themoviedb.org/movie/862-toy-story for tmdbId

In [74]:
dflinks.isnull().sum()

movieId    0
imdbId     0
tmdbId     8
dtype: int64

In [75]:
dflinks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9742 non-null   int64  
 1   imdbId   9742 non-null   int64  
 2   tmdbId   9734 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 228.5 KB


Let's get a dictionary with the most commom tags in each movie

In [76]:
dftags['tag']=dftags['tag'].str.lower() ## let's pull all of them in lowercase letters

In [77]:
pd.DataFrame(dftags['tag'].value_counts())

,tag
in netflix queue,131
atmospheric,41
thought-provoking,24
surreal,24
funny,24
...,...
turkey,1
mozart,1
out of order,1
celebrity fetishism,1


Let's just keep tha values with a frequency bigger than 10

In [78]:
mask = pd.DataFrame((dftags['tag'].value_counts())> 10)
to_keep=mask.index[mask['tag'] == True].tolist() ## This are the tags to keep for our model
to_keep

['in netflix queue',
 'atmospheric',
 'thought-provoking',
 'surreal',
 'funny',
 'superhero',
 'disney',
 'sci-fi',
 'religion',
 'quirky',
 'dark comedy',
 'suspense',
 'psychology',
 'visually appealing',
 'twist ending',
 'crime',
 'comedy',
 'politics',
 'music',
 'action',
 'high school',
 'time travel',
 'mental illness',
 'dark',
 'mindfuck',
 'aliens',
 'space',
 'dreamlike',
 'black comedy',
 'holocaust',
 'journalism',
 'shakespeare',
 'mafia',
 'stephen king',
 'disturbing',
 'heist',
 'emotional',
 'satire',
 'anime',
 'christmas',
 'classic',
 'court',
 'ghosts',
 'bittersweet',
 'comic book',
 'animation',
 'adolescence',
 'boxing',
 'psychological',
 'adultery',
 'imdb top 250']

In [79]:
# Lets use a get dummies to obtain a Onehotencoder result, then apply to_keep for only keeping the columns with a
# frequency bigger than 10 
dict_tags=pd.get_dummies(dftags['tag'])
dict_tags=dict_tags[to_keep]

In [80]:
# Lets add this to the original tags dataframe
dftags=dftags.join(dict_tags)

In [81]:
dftags.head()

,userId,movieId,tag,in netflix queue,atmospheric,thought-provoking,surreal,funny,superhero,disney,...,court,ghosts,bittersweet,comic book,animation,adolescence,boxing,psychological,adultery,imdb top 250
0,2,60756,funny,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,60756,highly quotable,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,60756,will ferrell,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2,89774,boxing story,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,89774,mma,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [82]:
## Then we could also do a group by movieId, so we have all the tags for a specific movie
tagsbymovie = dftags.drop('userId', axis=1).groupby('movieId').agg(lambda x: sum(x)).reset_index()
tagsbymovie.head()

,movieId,in netflix queue,atmospheric,thought-provoking,surreal,funny,superhero,disney,sci-fi,religion,...,court,ghosts,bittersweet,comic book,animation,adolescence,boxing,psychological,adultery,imdb top 250
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [83]:
dfmovies = dfmovies.merge(tagsbymovie, how='left').fillna(0.0)
dfmovies.head()

,movieId,title,year,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,court,ghosts,bittersweet,comic book,animation,adolescence,boxing,psychological,adultery,imdb top 250
0,1,Toy Story,1995,0,1,1,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,1995,0,1,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,1995,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,1995,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,1995,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Lets merge the ranking dt with our final movies/tags df:

In [84]:
df = dfratings.set_index('movieId').join(dfmovies.set_index('movieId')).reset_index()

In [85]:
df.head()

,movieId,userId,rating,dates,title,year,Action,Adventure,Animation,Children,...,court,ghosts,bittersweet,comic book,animation,adolescence,boxing,psychological,adultery,imdb top 250
0,1,1,4.0,2000-07-30 20:45:03,Toy Story,1995,0,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,5,4.0,1996-11-08 07:36:02,Toy Story,1995,0,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,7,4.5,2005-01-25 07:52:26,Toy Story,1995,0,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,15,2.5,2017-11-13 13:59:30,Toy Story,1995,0,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,17,4.5,2011-05-18 07:28:03,Toy Story,1995,0,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Non personalized recommendations 
- Optionel (Movies seen together) 
- Top 10 Movies (Semaine, mois, )
- Top 10 Movies per category (Semaine, mois, )
- Top 10 most popular (Semaine, mois, )
- Top 10 categories (Semaine, mois, )
- Top 10 tags (Semaine, mois, )

First, let's define a function that will consider only the data in a specific
period:

In [128]:
def dateperiod(df, days_past, end_date=datetime(2018, 9, 24)):    
    mask = ((df['dates'] > (end_date - timedelta(days=days_past))) & (df['dates'] <= end_date ))
    return df.loc[mask]

In [152]:
dateperiod(df, days_past=1)

,movieId,userId,rating,dates,title,year,Action,Adventure,Animation,Children,...,court,ghosts,bittersweet,comic book,animation,adolescence,boxing,psychological,adultery,imdb top 250
100779,187031,514,2.5,2018-09-23 05:55:27,Jurassic World: Fallen Kingdom,2018,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100800,187595,514,3.0,2018-09-23 05:55:46,Solo: A Star Wars Story,2018,1,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [160]:
df1 = dateperiod(df, days_past=7)
df_rating_mean = df1.groupby(['movieId','title'])['rating'].agg('mean').reset_index(name='rating_mean')
df_rating_count = df1.groupby(['movieId',])['rating'].agg('count').reset_index(name='rating_count')
dfTop10movies = df_rating_mean.merge(df_rating_count, on='movieId', how='left')
dfTop10movies = dfTop10movies[dfTop10movies['rating_count']>=10].sort_values(by = 'rating_mean',ascending = False).head(10)
dfTop10movies

,movieId,title,rating_mean,rating_count


In [161]:
dfTop10Popular = df1.groupby(['movieId','title'])['rating'].agg('count').reset_index(name='nb_ratings').sort_values(by='nb_ratings', ascending=False).head(10)
dfTop10Popular

,movieId,title,nb_ratings
180,187593,Deadpool 2,2
174,177765,Coco,2
0,10,GoldenEye,1
125,59369,Taken,1
116,54503,Superbad,1
117,54997,3:10 to Yuma,1
118,55765,American Gangster,1
119,55820,No Country for Old Men,1
120,56174,I Am Legend,1
121,56757,Sweeney Todd: The Demon Barber of Fleet Street,1


In [162]:
def top_popular(df, top, days_past):
    df1 = dateperiod(df, days_past=days_past)
    dfTop10Popular = df1.groupby(['movieId','title'])['rating'].agg('count').reset_index(name='nb_ratings').sort_values(by='nb_ratings', ascending=False).head(top)
    return dfTop10Popular

In [163]:
top_popular(df, 5, 10)

,movieId,title,nb_ratings
289,187593,Deadpool 2,3
189,106489,"Hobbit: The Desolation of Smaug, The",2
168,88140,Captain America: The First Avenger,2
109,35836,"40-Year-Old Virgin, The",2
193,106920,Her,2


In [164]:
def top_rating(df, top, days_past):
    df1 = dateperiod(df, days_past=days_past)
    
    df_rating_mean = df1.groupby(['movieId','title'])['rating'].agg('mean').reset_index(name='rating_mean')
    df_rating_count = df1.groupby(['movieId',])['rating'].agg('count').reset_index(name='rating_count')
    dfTop10movies = df_rating_mean.merge(df_rating_count, on='movieId', how='left')
    filter_factor = round(max(dfTop10movies['rating_count'])/3)
    dfTop10movies = dfTop10movies[dfTop10movies['rating_count']>filter_factor].sort_values(by = 'rating_mean',ascending = False).head(top)
    return dfTop10movies

In [166]:
top_rating(df, 5, 365)

,movieId,title,rating_mean,rating_count
96,318,"Shawshank Redemption, The",4.523810,21
3249,168252,Logan,4.500000,9
881,3578,Gladiator,4.461538,13
350,1258,"Shining, The",4.444444,9
82,260,Star Wars: Episode IV - A New Hope,4.406250,16
